# The Picture Frame Problem - Formulation 2

A company produces two types of picture frames: each type-1 frame uses 2 hours of labor and 1 ounce of metal and brings $2.25 in profit, and each type-2 frame uses 1 hour of labor and 2 ounces of metal and brings $2.60 in
profit. Each week 4000 labor hours and 5000 ounces of metal are available and the company aims to maximize its weekly profit with an optimal production plan.

To model this problem, let $x_1$ denote the number of type-1 frames produced and $x_2$ denote the number of type-2 frames produced. Then, this can be formulated as the following LP:

\begin{align*}
\underset{x}{\max} \quad &z = 2.25x_1 + 2.6 x_2 \\
\text{s.t.} \ \ \quad &2x_1 + x_2 \leq 4000, \\
&x_1 + 2x_2 \leq 5000, \\
&x \geq 0.
\end{align*}

Let's model this problem using JuMP.

### Problem Data

In [16]:
#Types of picture frames produced
types = [:type_1, :type_2]

#Hours of labor required for each type of frame
labor = Dict(:type_1 => 2, :type_2 => 1)

#Metal required for each type of frame (in ounces)
metal = Dict(:type_1 => 1, :type_2 => 2)

#Profit made from each type
profit = Dict(:type_1 => 2.25, :type_2 => 2.6)

#Quantity available of each resource
num_labor = 4000
num_metal = 5000;

@show metal

metal = Dict(:type_1 => 1, :type_2 => 2)


Dict{Symbol, Int64} with 2 entries:
  :type_1 => 1
  :type_2 => 2

In [17]:
#Import JuMP package to build an optimization model
using JuMP
#Import HiGHS solver
using HiGHS

#Create a JuMP model named picframe1 that will be solved using the HiGHS solver
picframe2 = Model(HiGHS.Optimizer);

#Add the variables frame[:type_1] and frame[:type_2] that are nonnegative
@variable(picframe2, frame[types] >= 0);

#Create the constraints, name them
@constraint(picframe2, max_labor, sum(labor[i]*frame[i] for i in types) <= num_labor);
@constraint(picframe2, max_metal, sum(metal[i]*frame[i] for i in types) <= num_metal);

#Create our objective function and set it for minimization
@objective(picframe2, Max, sum(profit[i]*frame[i] for i in types));

print(picframe2)

In [18]:
#Solve the model
optimize!(picframe2);
#Outputs detailed information about the solution process
@show solution_summary(picframe2);

Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Presolve : Reductions: rows 2(-0); columns 2(-0); elements 4(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.8499947627e+00 Ph1: 2(6); Du: 2(4.84999) 0s
          2    -7.4500000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  7.4500000000e+03
HiGHS run time      :          0.00
solution_summary(picframe2) = * Solver : HiGHS

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution
  Objective value      : 7.45000e+03
  Objective bound      : 7.45000e+03
  Relative gap         : Inf
  Dual objective value : 7.45000e+03

* Work counters
  Solve time (sec)   : 9.87291e-04
  Simplex ite

In [19]:
#Status of primal problem
@show primal_status(picframe2);
#Status of dual problem
@show dual_status(picframe2);
#Final objective value
@show objective_value(picframe2);
#Value of type 1 at solution
@show value(frame[:type_1]);
#Value of type 2 at solution
@show value(frame[:type_2]);
#Shadow price of labor constraint (value of dual variable for labor) at solution
@show shadow_price(max_labor);
#Shadow price of metal constraint (value of dual variable for metal) at solution
@show shadow_price(max_metal);

primal_status(picframe2) = MathOptInterface.FEASIBLE_POINT
dual_status(picframe2) = MathOptInterface.FEASIBLE_POINT
objective_value(picframe2) = 7450.0
value(frame[:type_1]) = 1000.0
value(frame[:type_2]) = 2000.0
shadow_price(max_labor) = 0.6333333333333333
shadow_price(max_metal) = 0.9833333333333334


We can see that the optimal solution matches the solution we got from the first formulation.